<h1>OpenAI Tool Usage</h1>

<h3>Objectives:</h3>
<p>
<ol>
<li>Tool Usage</li>
<li>Gradio + streaming</li>
</ol>
</p>

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")


OpenAI API Key exists and begins sk-axLiE


In [3]:
# Initialize

openai = OpenAI()

In [4]:
# defining common system message
system_message = "You are a polite and helpful application support assistant for an application called 'Fruit Basket'. "
system_message += "You are to help users with issues they experience while progressing quotes on the 'Fruit Basket' application. "
system_message += "All quotes have an associated quoteId to uniquely identify the quote. "
system_message += "If user has not provided the quoteId, ask the user to provide the same. "
system_message += "If user asks anything except assistance with 'Fruit Basket' quote, then politely decline the request "
system_message += "and ask user to reach out to application_support@fruitbasket.com. "

In [5]:
# Open and read the JSON file
with open('data.json', 'r') as file:
    data = json.load(file)

# Print the 2nd record of data
print(data['quotes'][1])


{'quoteId': 'FB002', 'quoteStage': 'quote config', 'quoteStageStatus': 'config pending', 'terminalStatus': 'FALSE', 'description': 'User needs to add configuration against chosen product(s)'}


In [6]:
# function to retrieve quote details
def quote_details(quoteId):
    result = {'quoteId': 'NA', 'quoteStage': 'NA', 'quoteStageStatus': 'NA', 'terminalStatus': 'FALSE', 'description': 'Quote Not Found'}
    for i in range(len(data['quotes'])):
        if quoteId in data['quotes'][i].get('quoteId'):
            result = data['quotes'][i]
            break
    return result


In [7]:
# test function - incorrect quote id
quote_details('asdas')

{'quoteId': 'NA',
 'quoteStage': 'NA',
 'quoteStageStatus': 'NA',
 'terminalStatus': 'FALSE',
 'description': 'Quote Not Found'}

In [8]:
# test function
quote_details('FB012')

{'quoteId': 'FB012',
 'quoteStage': 'customize',
 'quoteStageStatus': 'customize incomplete',
 'terminalStatus': 'FALSE',
 'description': 'User has added incomplete product customization. Ask user to complete the product customization'}

In [9]:
# defining the metadata for quote_details function using the pre-defined dict structure
quote_function = {
    "name": "quote_details",
    "description": "Get the quote details of user's quote. Call this whenever details of user's quote needs to be fetched. For example, when a customer asks 'Why is my quote FB002 stuck?'",
    "parameters": {
        "type": "object",
        "properties": {
            "quoteId": {
                "type": "string",
                "description": "The quoteId that needs to be searched",
            },
        },
        "required": ["quoteId"],
        "additionalProperties": False
    }
}


In [10]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": quote_function}]

In [11]:
MODEL = "gpt-4o-mini"

In [12]:
# chat function with tool call - NON-STREAMING VERSION
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, description = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    #print(messages)
    return response.choices[0].message.content

In [13]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    quoteId = arguments.get('quoteId')
    quote = quote_details(quoteId)
    response = {
        "role": "tool",
        "content": json.dumps(quote),
        "tool_call_id": tool_call.id
    }
    return response, quote.get('description')

In [14]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [15]:
# handle_stream_tool_call definition
def handle_stream_tool_call(final_tool_calls):
    tool_call = final_tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    quoteId = arguments.get('quoteId')
    quote = quote_details(quoteId)
    response = {
        "role": "tool",
        "content": json.dumps(quote),
        "tool_call_id": tool_call.id
    }
    return response, quote.get('description')

In [16]:
# chat function with tool call
def chat_stream(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    stream = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools, stream=True)

    response = ""
    final_tool_calls = {}
    
    for chunk in stream:
        if chunk.choices[0].delta.tool_calls and len(chunk.choices[0].delta.tool_calls) > 0:
            for tool_call in chunk.choices[0].delta.tool_calls or []:
                index = tool_call.index
                if index not in final_tool_calls:
                    final_tool_calls[index] = tool_call
                final_tool_calls[index].function.arguments += tool_call.function.arguments
                response += "processing..."
        elif chunk.choices[0].finish_reason=="tool_calls":
            #print(final_tool_calls)
            response, description = handle_stream_tool_call(final_tool_calls)
            assistant_message = {"role":"assistant", "tool_calls":[
                {"id":final_tool_calls[0].id, "function":final_tool_calls[0].function, "type": final_tool_calls[0].type}
            ]}
            
            #print(assistant_message)
            #print(response)
            messages.append(assistant_message)
            messages.append(response)
            #print(messages)
            tool_stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)
            response = ""
            for new_chunk in tool_stream:
                response += new_chunk.choices[0].delta.content or ''
                yield response
        else:
            response += chunk.choices[0].delta.content or ''
        yield response


In [17]:
gr.ChatInterface(fn=chat_stream, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
